### Thuật Toán CURE

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.utils import resample
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D 


### Đọc dữ liệu

In [13]:
# Đọc dữ liệu
data = pd.read_csv('student_depression_dataset.csv')
print(data)

           id  Gender   Age           City Profession  Academic Pressure  \
0           2    Male  33.0  Visakhapatnam    Student                5.0   
1           8  Female  24.0      Bangalore    Student                2.0   
2          26    Male  31.0       Srinagar    Student                3.0   
3          30  Female  28.0       Varanasi    Student                3.0   
4          32  Female  25.0         Jaipur    Student                4.0   
...       ...     ...   ...            ...        ...                ...   
27896  140685  Female  27.0          Surat    Student                5.0   
27897  140686    Male  27.0       Ludhiana    Student                2.0   
27898  140689    Male  31.0      Faridabad    Student                3.0   
27899  140690  Female  18.0       Ludhiana    Student                5.0   
27900  140699    Male  27.0          Patna    Student                4.0   

       Work Pressure  CGPA  Study Satisfaction  Job Satisfaction  \
0                0.

### Quy Ước 

In [14]:

# --- 1. Chuẩn hóa cột Gender ---
data['Gender'] = data['Gender'].astype(str).str.strip().str.capitalize()
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})

# --- 2. Chuẩn hóa cột Dietary Habits ---
data['Dietary Habits'] = data['Dietary Habits'].astype(str).str.strip().str.capitalize()
data['Dietary Habits'] = data['Dietary Habits'].map({
    'Unhealthy': 0,
    'Moderate': 1,
    'Healthy': 2,
    'Others': 3 
})

# --- 3. Chuẩn hóa cột "Have you ever had suicidal thoughts ?" ---
# Kiểm tra đúng tên cột (nên copy y nguyên)
suicidal_col = "Have you ever had suicidal thoughts ?"
data[suicidal_col] = data[suicidal_col].astype(str).str.strip().str.capitalize()
data[suicidal_col] = data[suicidal_col].map({'No': 0, 'Yes': 1})

# Quy ước Family History of Mental Illness: 0 - No, 1 - Yes
data['Family History of Mental Illness'] = data['Family History of Mental Illness'].map({
    'No': 0,
    'Yes': 1
})

# Kiểm tra kết quả
print(data.head())

   id  Gender   Age           City Profession  Academic Pressure  \
0   2       0  33.0  Visakhapatnam    Student                5.0   
1   8       1  24.0      Bangalore    Student                2.0   
2  26       0  31.0       Srinagar    Student                3.0   
3  30       1  28.0       Varanasi    Student                3.0   
4  32       1  25.0         Jaipur    Student                4.0   

   Work Pressure  CGPA  Study Satisfaction  Job Satisfaction  \
0            0.0  8.97                 2.0               0.0   
1            0.0  5.90                 5.0               0.0   
2            0.0  7.03                 5.0               0.0   
3            0.0  5.59                 2.0               0.0   
4            0.0  8.13                 3.0               0.0   

        Sleep Duration  Dietary Habits   Degree  \
0          '5-6 hours'               2  B.Pharm   
1          '5-6 hours'               1      BSc   
2  'Less than 5 hours'               2       BA   
3 

### Xử lý Sleep Duration

In [15]:
import pandas as pd
import re

def convert_sleep_duration(duration_str):
    if pd.isna(duration_str):
        return None

    duration_str = str(duration_str).lower().strip()

    # Trường hợp "less than X hours"
    match_less = re.search(r"less than (\d+\.?\d*)", duration_str)
    if match_less:
        hour = float(match_less.group(1))
        return hour - 0.5

    # Trường hợp "more than X hours"
    match_more = re.search(r"more than (\d+\.?\d*)", duration_str)
    if match_more:
        hour = float(match_more.group(1))
        return hour + 0.5

    # Trường hợp "X-Y hours"
    match_range = re.findall(r"(\d+\.?\d*)", duration_str)
    if len(match_range) == 2:
        return (float(match_range[0]) + float(match_range[1])) / 2

    # Trường hợp "X hours"
    match_single = re.search(r"(\d+\.?\d*)", duration_str)
    if match_single:
        return float(match_single.group(1))

    return None

def categorize_sleep_duration(hours):
    if pd.isna(hours):
        return None
    elif hours < 6:
        return 0
    elif 6 <= hours <= 8:
        return 1
    else:
        return 2

# Áp dụng chuyển đổi
data['Sleep Duration'] = data['Sleep Duration'].apply(convert_sleep_duration)
data['Sleep Duration'] = data['Sleep Duration'].apply(categorize_sleep_duration)

print(data.head(10))


   id  Gender   Age           City Profession  Academic Pressure  \
0   2       0  33.0  Visakhapatnam    Student                5.0   
1   8       1  24.0      Bangalore    Student                2.0   
2  26       0  31.0       Srinagar    Student                3.0   
3  30       1  28.0       Varanasi    Student                3.0   
4  32       1  25.0         Jaipur    Student                4.0   
5  33       0  29.0           Pune    Student                2.0   
6  52       0  30.0          Thane    Student                3.0   
7  56       1  30.0        Chennai    Student                2.0   
8  59       0  28.0         Nagpur    Student                3.0   
9  62       0  31.0         Nashik    Student                2.0   

   Work Pressure  CGPA  Study Satisfaction  Job Satisfaction  Sleep Duration  \
0            0.0  8.97                 2.0               0.0             0.0   
1            0.0  5.90                 5.0               0.0             0.0   
2          

### Loại bỏ những cột không cần thiết

In [16]:
cols_to_drop = [
    'id',
    'City',
    'Profession',
    'Work Pressure',
    'Job Satisfaction',
    'Degree',
    'Depression'  # Chỉ bỏ nếu bạn không phân tích nhãn nữa
]

# Xoá cột
data_cleaned = data.drop(columns=cols_to_drop)

print(data_cleaned.head())
print(data_cleaned.columns)

   Gender   Age  Academic Pressure  CGPA  Study Satisfaction  Sleep Duration  \
0       0  33.0                5.0  8.97                 2.0             0.0   
1       1  24.0                2.0  5.90                 5.0             0.0   
2       0  31.0                3.0  7.03                 5.0             0.0   
3       1  28.0                3.0  5.59                 2.0             1.0   
4       1  25.0                4.0  8.13                 3.0             0.0   

   Dietary Habits  Have you ever had suicidal thoughts ?  Work/Study Hours  \
0               2                                      1               3.0   
1               1                                      0               3.0   
2               2                                      0               9.0   
3               1                                      1               4.0   
4               1                                      1               1.0   

  Financial Stress  Family History of Mental Illne

### Kiểm tra dữ liệu trùng lặp

In [17]:
print("Số dòng trước khi loại bỏ:", len(data))
df = data_cleaned.drop_duplicates()
print("Số dòng sau khi loại bỏ:", len(df))

Số dòng trước khi loại bỏ: 27901
Số dòng sau khi loại bỏ: 27897


### Lọc chỉ lấy các cột số

In [18]:
df["Financial Stress"] = df["Financial Stress"].replace("?", np.nan)
df["Financial Stress"] = df["Financial Stress"].astype(float)

# Chọn ra các cột số cần chuẩn hoá
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Nếu có NaN, bạn có thể điền giá trị trung bình vào các NaN
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Khởi tạo scaler và fit-transform
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Kiểm tra lại dữ liệu sau khi chuẩn hóa
print(df[numeric_cols].head())

# Giảm từ 11 chiều xuống 3 chiều
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df[numeric_cols])  # X_scaled là dữ liệu đã chuẩn hóa

# Kiểm tra kết quả PCA
print(X_pca)


     Gender       Age  Academic Pressure      CGPA  Study Satisfaction  \
0 -0.891531  1.463238           1.345534  0.893410           -0.693332   
1  1.121666 -0.371440          -0.826122 -1.193993            1.510619   
2 -0.891531  1.055532          -0.102237 -0.425666            1.510619   
3  1.121666  0.443972          -0.102237 -1.404773           -0.693332   
4  1.121666 -0.167587           0.621648  0.322263            0.041319   

   Sleep Duration  Dietary Habits  Have you ever had suicidal thoughts ?  \
0       -0.868362        1.371751                               0.761714   
1       -0.868362        0.118591                              -1.312828   
2       -0.868362        1.371751                              -1.312828   
3        0.377473        0.118591                               0.761714   
4       -0.868362        0.118591                               0.761714   

   Work/Study Hours  Financial Stress  Family History of Mental Illness  
0         -1.121207     

C:\Users\DUC\AppData\Local\Temp\ipykernel_20072\374062933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Financial Stress"] = df["Financial Stress"].replace("?", np.nan)
C:\Users\DUC\AppData\Local\Temp\ipykernel_20072\374062933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Financial Stress"] = df["Financial Stress"].astype(float)
C:\Users\DUC\AppData\Local\Temp\ipykernel_20072\374062933.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

### Check các cụm qtrong

In [19]:
import pandas as pd
from sklearn.decomposition import PCA

# Giả sử bạn đã chuẩn hóa và giảm chiều dữ liệu với PCA
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Khởi tạo PCA và thực hiện giảm chiều
pca = PCA(n_components=2)
X_pca = pca.fit_transform(df[numeric_cols])

# Lấy các trọng số của từng cột trong các thành phần chính (components_)
components_df = pd.DataFrame(pca.components_, columns=numeric_cols)

# Để tìm tên cột có ảnh hưởng lớn nhất đến mỗi thành phần chính:
for i in range(components_df.shape[0]):
    print(f"Principal Component {i+1}:")
    print(components_df.iloc[i].sort_values(ascending=False).head(3))  # 3 cột ảnh hưởng nhất
    print("\n")


Principal Component 1:
Have you ever had suicidal thoughts ?    0.536832
Academic Pressure                        0.488439
Financial Stress                         0.431947
Name: 0, dtype: float64


Principal Component 2:
Gender                0.678894
Dietary Habits        0.288589
Study Satisfaction    0.256324
Name: 1, dtype: float64




### CURE

In [20]:
class CURE:
    def __init__(self, k, a, num_representatives):
        self.k = k  # Số cụm mong muốn
        self.shrink_factor = a  # Hệ số co rút
        self.num_representatives = num_representatives  # Số điểm đại diện của mỗi cụm
        self.clusters = []  # Danh sách chứa các cụm
        self.labels_ = None  # Nhãn của từng điểm
    
    def fit(self, df):
        # Kiểm tra số lượng mẫu ban đầu
        print(f"Số mẫu ban đầu trong DataFrame: {len(df)}")

        if len(df) > 2500:
            df = resample(df, n_samples=2500, random_state=42)
            print(f"Số mẫu sau khi lấy ngẫu nhiên: {len(df)}")
            
        # Chọn các cột số và lấy giá trị dưới dạng mảng numpy
        numeric_cols = ['Gender', 'Age', 'Academic Pressure', 'CGPA', 'Study Satisfaction','Sleep Duration', 'Dietary Habits','Have you ever had suicidal thoughts ?', 'Work/Study Hours','Financial Stress', 'Family History of Mental Illness']
        X = df[numeric_cols].values
        self.X_ = X

        self.clusters = [[x] for x in X]  # Mỗi điểm bắt đầu là một cụm riêng
        data_map = {tuple(x): idx for idx, x in enumerate(X)}  # Lưu vị trí ban đầu của từng điểm

        # Bước 1: Hợp nhất các cụm gần nhất
        while len(self.clusters) > self.k:
            min_dist = float('inf')
            merge_idx = (-1, -1)

            for i in range(len(self.clusters)):
                for j in range(i + 1, len(self.clusters)):
                    dist = np.min(cdist(self.clusters[i], self.clusters[j]))
                    if dist < min_dist:
                        min_dist = dist
                        merge_idx = (i, j)

            i, j = merge_idx
            self.clusters[i].extend(self.clusters[j])
            del self.clusters[j]

        # Gán nhãn cho từng điểm dữ liệu
        self.labels_ = np.zeros(len(X), dtype=int)
        for cluster_idx, cluster in enumerate(self.clusters):
            for point in cluster:
                idx = data_map.get(tuple(point))
                if idx is not None:
                    self.labels_[idx] = cluster_idx

        # Kiểm tra điểm trong mỗi cụm
        for i, cluster in enumerate(self.clusters):
            print(f"Cụm {i+1} có {len(cluster)} điểm")

        # Bước 3: Tính điểm đại diện mới 
        self.representatives = []
        for cluster in self.clusters:
            if len(cluster) > self.num_representatives:
                farthest_points = self._get_farthest_points(cluster)
                centroid = np.mean(farthest_points, axis=0)  # Tính trung tâm cụm
                shrunk_points = centroid + self.shrink_factor * (farthest_points - centroid)  # Điểm đại diện mới
                self.representatives.append(shrunk_points)
            else:
                self.representatives.append(np.array(cluster))

    # Bước 2: Lấy điểm xa nhất 
    def _get_farthest_points(self, cluster):
        cluster = np.array(cluster)
        centroid = np.mean(cluster, axis=0)
        distances = np.linalg.norm(cluster - centroid, axis=1)
        return cluster[np.argsort(distances)[-self.num_representatives:]]

    
    def plot_clusters(self):
        plt.figure(figsize=(10, 8))
        colors = plt.cm.get_cmap('Set1', self.k)  # Bảng màu rõ ràng

        # Vẽ các điểm dữ liệu theo cụm
        for i in range(self.k):
            cluster_points = self.X_[self.labels_ == i]
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1],
                color=colors(i), label=f'Cụm {i+1}', alpha=0.6)

        # Vẽ các điểm đại diện
        for reps in self.representatives:
            reps = np.array(reps)
            plt.scatter(reps[:, 0], reps[:, 1],
                color='yellow', edgecolors='black',
                marker='X', s=200, linewidths=1.5, label='Điểm đại diện cụm')

        # Tiêu đề và nhãn trục
        plt.title('CURE Clustering', fontsize=14)
        plt.xlabel('Have you ever had suicidal thoughts ?')
        plt.ylabel('Gender')

        # Đánh giá cụm
        from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
        silhouette = silhouette_score(self.X_, self.labels_)
        davies = davies_bouldin_score(self.X_, self.labels_)
        calinski = calinski_harabasz_score(self.X_, self.labels_)

        # Hiển thị thông số đánh giá ở góc trên bên trái
        plt.text(0.01, 0.99,
                f'Silhouette: {silhouette:.4f}\nDavies-Bouldin: {davies:.4f}\nCalinski-Harabasz: {calinski:.1f}',
                transform=plt.gca().transAxes,
                fontsize=11, verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.7))

        plt.legend()
        plt.tight_layout()
        plt.show()


### ELBOW METHOD

In [21]:
# results = []

# for k in range(2, 10):  # Thử từ 2 đến 10 cụm
#     cure = CURE(k=k, a=0.3, num_representatives=5)
#     cure.fit(df)  # Huấn luyện mô hình
#     labels = cure.labels_  # Lấy nhãn của các điểm dữ liệu
#     score = silhouette_score(df, labels)
    
#     results.append({"Số cụm (k)": k, "Silhouette Score": score})

# # Tạo DataFrame
# df_results = pd.DataFrame(results).sort_values(by="Silhouette Score", ascending=False)

# # Hiển thị bảng
# from IPython.display import display
# display(df_results)


In [22]:
cure = CURE(k=2, a=0.3, num_representatives=5)
cure.fit(df)
cure.plot_clusters()

Số mẫu ban đầu trong DataFrame: 27897
Số mẫu sau khi lấy ngẫu nhiên: 2500


KeyboardInterrupt: 